        %reload_ext deathbeds.__SyntaxError__
        
## Both

* Are `nbformat` cells & have `metadata` _have a schema_.
* Accept text input
* Contain block code
* Create rich displays
* Are salient to the document

    

For reusability, it is important that code works.  A computational essay should promote a 
deeper entanglement between code types.

## Markdown

* Rich display replaces the input cell.
* There are no syntax errors.
* Does not run code.
* `#` means something 
* In command mode, `M` toggles to `Markdown` cells 
* Supplements the contents of `Code` cells.


## Code

* Rich display shows after the input cell.
* May raise `SyntaxError` or `IndentationError` that indicate bad inputs.
* Runs code.
* `#` means something _un_important
* In command mode, `Y` toggles to `Code` cells
* Supplements the contents of `Markdown` cells.

In [1]:
    from deathbeds.__Markdown_code_cells import CallableTransformer, CodeRenderer
    import ast; from textwrap import dedent; import fnmatch

## Markdown documents

### Code Blocks

On [2018-07-06-Markdown-code-cells.ipynb](2018-07-06-Markdown-code-cells.ipynb) we discussed making runnable Markdown code cells.  This document only consider the block code.  The current 
`CodeRenderer` we use is able to created block code for us.

In [2]:
    class InlineCodeCellMagicException(SyntaxError): ...
    class NoMarkdownCodeException(SyntaxError):...

# Inline code

Inline code is evaluated after block code.

In [3]:
    class Register:
        def __enter__(self): get_ipython().input_transformer_manager.physical_line_transforms = [self] + self.__exit__()

        def __exit__(self, *_, **__):
            get_ipython().input_transformer_manager.physical_line_transforms = [
                object for object in get_ipython().input_transformer_manager.physical_line_transforms
                if not isinstance(object, type(self))]
            return get_ipython().input_transformer_manager.physical_line_transforms


In [13]:
    class SyntaxErrorTransform(ast.NodeTransformer, CodeRenderer, CallableTransformer, Register):
        def code(self, node, entering):  self.nodes.append(node)

        def __call__(self, str):
            self.nodes = []
            current, lines, str = 0, str.splitlines(), CodeRenderer.__call__(self, str)
                        
            if str.lstrip().startswith('%%') and self.nodes: 
                raise InlineCodeCellMagicException("""Inline code may not be mixed with cell magics.""") from Exception                
            if lines and not (str.strip() or self.nodes): raise NoMarkdownCodeException(
                'No code object found in the markdown.') from Exception

            str = (str or '\n').splitlines()
            
            while """Increment the lines until there are enough of them.
            """ and (len(str) < len(lines)): str.append("")

            for node in self.nodes:
                for current in range(current, len(lines)):
                    if fnmatch.fnmatch(lines[current], f"*`{node.literal}`*"): break
                if str[current].endswith('\\'): str[current] = str[current].rstrip('\\') + node.literal
                elif str[current]: str[current] += ';' + node.literal
                else: str[current] = node.literal
            return dedent('\n'.join(str))

In [15]:
    errors = SyntaxError, IndentationError

`markdown_trap` temporarily replaces the input transformers to accept Markdown as source code when the `errors` are trigger.

In [1]:
    def markdown_trap(self, Exception, object, traceback, **dict):
        unload_ipython_extension(self)
        with SyntaxErrorTransform(): self.run_cell(In[-1]), load_ipython_extension(self)

In [16]:
    def load_ipython_extension(ip): ip.set_custom_exc(errors, markdown_trap)
    def unload_ipython_extension(ip):  ip.set_custom_exc(tuple(), None)

In [17]:
__name__ == '__main__' and load_ipython_extension(get_ipython())

A key to literate computing is to ease the transition be authoring human and computer readable documnets.

The `range(\` starts at `0,\` and ends at `10)` .

## Advantages of literate code cells